# DATA20001 Deep Learning - Group Project
## Image project

**Due Thursday, May 20, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imread` from `matplotlib.pyplot`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Get the data

In [2]:
import sys
import os
from os.path import join
from os.path import abspath
from os.path import split

import torch
import torchvision
from torchvision.datasets.utils import download_url
import zipfile

import numpy as np
import pandas as pd
from matplotlib.pyplot import imread

root_dir = os.getcwd()
if root_dir not in sys.path:
    sys.path.append(root_dir)

train_path = 'train'
test_path = 'test'

data_zip_name = 'dl2018-image-proj.zip'
DATA_ZIP_DIR = os.path.abspath(os.path.join(root_dir, data_zip_name))

with zipfile.ZipFile(DATA_ZIP_DIR) as zip_f:
    zip_f.extractall(train_path)

In [78]:
# create pd dataframe to store mapping from image to all labels easily
# Check this out to create custome data loader: https://github.com/utkuozbulak/pytorch-custom-dataset-examples
path = 'annotations/'
filenames = os.listdir(path)
res = {"filename": [f"im{i}.jpg" for i in range(20001)]}

for fname in filenames:
    _name = fname[:-4]
    data = open(path+fname, "r").read().splitlines()
    data = np.array(list(map(int, data)))
    np_vec = np.zeros(20001)
    np_vec[data] = 1
    res[_name] = np_vec

df = pd.DataFrame(res)
df.drop(0, axis=0, inplace=True)
df.to_csv("file_to_labels_table.csv", index=False)
df.head()

,filename,clouds,male,bird,dog,river,portrait,baby,night,people,female,sea,tree,car,flower
1,im1.jpg,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,im2.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,im3.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,im4.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,im5.jpg,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [80]:
df = pd.read_csv("file_to_labels_table.csv")
tmp = df.sum(axis=1)
zeroes = (tmp==0)
nonzeroes = (tmp!=0)

print(df.shape)
print(nonzeroes.sum(), zeroes.sum(), nonzeroes.sum()+zeroes.sum())
print(df.sum(axis=0))

(20000, 15)
10176 9824 20000
filename    im1.jpgim2.jpgim3.jpgim4.jpgim5.jpgim6.jpgim7....
clouds                                                   1095
male                                                     2979
bird                                                      360
dog                                                       448
river                                                     120
portrait                                                 3121
baby                                                       95
night                                                     598
people                                                   6403
female                                                   3227
sea                                                       173
tree                                                      525
car                                                       319
flower                                                    761
dtype: object


In [92]:
# shuffle array and perform train/test split
arr = np.arange(20000)
np.random.shuffle(arr)

# first 90% for training, remaining 10% for testing
split_index = int(arr.shape[0]*.9)
print("split index", split_index)
train_idx = arr[:split_index]
test_idx = arr[split_index:]
print("index lengths:", train_idx.shape[0], test_idx.shape[0])
assert len(np.intersect1d(train_idx, test_idx)) == 0, "The two sets should be mutually exclusive"

# take the subset of the label data for each dataset
labels_train = df.iloc[train_idx, :]
labels_test = df.iloc[test_idx, :]
print(labels_test.sum(axis=0))

split index 18000
index lengths: 18000 2000
[]
filename    im18995.jpgim4067.jpgim4819.jpgim15437.jpgim73...
clouds                                                    114
male                                                      298
bird                                                       37
dog                                                        38
river                                                      13
portrait                                                  298
baby                                                       11
night                                                      51
people                                                    660
female                                                    314
sea                                                        15
tree                                                       61
car                                                        36
flower                                                     69
dtype: object


In [33]:
path = 'images/'
filenames = os.listdir(path)
# vec = imread("images/im20000.jpg")
color_count = 0
grey_count = 0
for i in range(1, 20001):
    vec = imread(f"images/im{i}.jpg")
    if (vec.shape == (128, 128, 3)):
        color_count += 1
    elif (vec.shape == (128, 128)):
        grey_count += 1
    else:
        raise NotImplemented("da fugelis")
print(color_count)
print(grey_count)
#for fname in filenames:
#    with open(path+fname, "r") as f:
#        data = f.readlines()
#        all_labels.extend(data)
#        counter += len(data)
#print(type(vec))
#print(len(filenames))

18578
1422


The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Your stuff goes here ...

In [ ]:
# Here goes your code ...

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".  [See more here on how to load the model back](https://github.com/pytorch/pytorch/blob/761d6799beb3afa03657a71776412a2171ee7533/docs/source/notes/serialization.rst) if you want to continue training later.

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')